# Fourier Terms for Seasonality

When a series has strong periodic structure, sine/cosine terms provide a compact way to model
seasonality inside linear or tree-based models.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(21)

n = 180
t = np.arange(n)
period = 30
y = 5 + 0.03 * t + 2.2 * np.sin(2 * np.pi * t / period) + 0.8 * np.cos(2 * np.pi * t / period)
y = y + np.random.normal(0, 0.5, n)
series = pd.Series(y, index=pd.RangeIndex(n), name="y")

series.head()


## Fourier feature construction

For seasonal period $S$ and $K$ Fourier pairs, define:

$$x_{t,k}^{(sin)} = \sin\left(
\frac{2\pi k t}{S}
\right), \quad x_{t,k}^{(cos)} = \cos\left(
\frac{2\pi k t}{S}
\right), \quad k = 1,\dots,K$$

These features can be used in linear regression or in a forecasting pipeline.


In [ ]:
K = 3
X = []
for k in range(1, K + 1):
    X.append(np.sin(2 * np.pi * k * t / period))
    X.append(np.cos(2 * np.pi * k * t / period))
X = np.column_stack(X)

# Add bias and trend term
X_design = np.column_stack([np.ones(n), t, X])
beta, *_ = np.linalg.lstsq(X_design, series.values, rcond=None)
y_hat = X_design @ beta


## Fit with Fourier terms

A small number of Fourier pairs often captures most of the seasonal structure.


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=series, mode="lines", name="y_t", line=dict(color="#1f77b4")))
fig.add_trace(go.Scatter(x=series.index, y=y_hat, mode="lines", name="Fourier fit", line=dict(color="#ff7f0e")))
fig.update_layout(title="Fourier term reconstruction", xaxis_title="t", yaxis_title="value", height=420)
fig.show()


## Basis functions

The first few sine/cosine pairs form a smooth basis for seasonality.


In [ ]:
basis_fig = go.Figure()
for k in range(1, 4):
    basis_fig.add_trace(go.Scatter(x=t[:period], y=np.sin(2 * np.pi * k * t[:period] / period), mode="lines", name=f"sin k={k}"))
    basis_fig.add_trace(go.Scatter(x=t[:period], y=np.cos(2 * np.pi * k * t[:period] / period), mode="lines", name=f"cos k={k}", line=dict(dash="dash")))
basis_fig.update_layout(title="Fourier basis terms over one period", xaxis_title="t", yaxis_title="value", height=420)
basis_fig.show()


## Practical notes

- Larger $K$ captures sharper seasonal shapes but can overfit.
- Fourier terms are common in TBATS and regression-based forecasters.
- In sktime, `FourierFeatures` provides a ready-made transformer for this idea.
